## Basic prompting

In [14]:
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
from langchain.agents import create_agent
from langchain.messages import HumanMessage

#agent = create_agent(model="gpt-5-nano")

from langchain_nvidia_ai_endpoints import ChatNVIDIA

model = ChatNVIDIA(model="qwen/qwen3-next-80b-a3b-thinking")
agent = create_agent(model=model )


question = HumanMessage(content="What's the capital of the moon?")

response = agent.invoke(
    {"messages": [question]}
)

print(response['messages'][1].content)

That's a fun question—but the moon **doesn't have a capital**, and here's why:  

### 🌙 The Moon Isn't a Country or Sovereign Entity  
- The moon is a **natural satellite** of Earth, not a nation, territory, or independent political entity.  
- No country or government "owns" the moon. This is established by the **Outer Space Treaty of 1967** (ratified by over 110 countries), which states that celestial bodies like the moon cannot be claimed by any single nation.  

### 🚀 No Permanent Human Settlements Exist  
- Humans have only visited the moon briefly during NASA's **Apollo missions** (1969–1972), with the last crewed landing in 1972.  
- Today, there are **no permanent human habitats** on the moon. All missions (like recent robotic landers or short-term astronaut stays) are temporary.  
- While future missions (e.g., NASA's **Artemis program**) aim to build sustainable lunar bases, these would be scientific outposts—not cities with capitals.  

### 💡 Why This Question Comes Up  
Som

In [16]:
system_prompt = "You are a science fiction writer, create a capital city at the users request."

scifi_agent = create_agent(
    model=model,
    system_prompt=system_prompt
)

response = scifi_agent.invoke(
    {"messages": [question]}
)

print(response['messages'][1].content)

## Few-shot examples

In [ ]:
system_prompt = """

You are a science fiction writer, create a space capital city at the users request.

User: What is the capital of mars?
Scifi Writer: Marsialis

User: What is the capital of Venus?
Scifi Writer: Venusovia

"""

scifi_agent = create_agent(
    model="gpt-5-nano",
    system_prompt=system_prompt
)

response = scifi_agent.invoke(
    {"messages": [question]}
)

print(response['messages'][1].content)

## Structured prompts

In [ ]:
system_prompt = """

You are a science fiction writer, create a space capital city at the users request.

Please keep to the below structure.

Name: The name of the capital city

Location: Where it is based

Vibe: 2-3 words to describe its vibe

Economy: Main industries

"""

scifi_agent = create_agent(
    model="gpt-5-nano",
    system_prompt=system_prompt
)

response = scifi_agent.invoke(
    {"messages": [question]}
)

print(response['messages'][1].content)

## Structured output

In [10]:
tool_models = [model for model in ChatNVIDIA.get_available_models() if model.supports_structured_output and model.supports_tools]
for elem in tool_models:
    print(elem)

id='moonshotai/kimi-k2-instruct' model_type='chat' client='ChatNVIDIA' endpoint=None aliases=None supports_tools=True supports_structured_output=True supports_thinking=False thinking_prefix=None no_thinking_prefix=None thinking_param_enable=None thinking_param_disable=None base_model=None
id='meta/llama-3.1-405b-instruct' model_type='chat' client='ChatNVIDIA' endpoint=None aliases=None supports_tools=True supports_structured_output=True supports_thinking=False thinking_prefix=None no_thinking_prefix=None thinking_param_enable=None thinking_param_disable=None base_model=None
id='nv-mistralai/mistral-nemo-12b-instruct' model_type='chat' client='ChatNVIDIA' endpoint=None aliases=None supports_tools=True supports_structured_output=True supports_thinking=False thinking_prefix=None no_thinking_prefix=None thinking_param_enable=None thinking_param_disable=None base_model=None
id='moonshotai/kimi-k2-instruct-0905' model_type='chat' client='ChatNVIDIA' endpoint=None aliases=None supports_tools=

In [20]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA

# Fetch all available models from the NVIDIA endpoint
all_models = ChatNVIDIA.get_available_models()

# Filter for models specifically built for agentic workflows
agent_ready_models = [
    model.id for model in all_models 
    if model.model_type == "chat" 
    and model.supports_tools 
    and model.supports_structured_output
]

print("Models fully optimized for structured agents:")
for m in agent_ready_models:
    print(f"- {m}")

Models fully optimized for structured agents:
- moonshotai/kimi-k2-instruct
- meta/llama-3.1-405b-instruct
- nv-mistralai/mistral-nemo-12b-instruct
- moonshotai/kimi-k2-instruct-0905
- nvidia/nvidia-nemotron-nano-9b-v2
- meta/llama-3.1-70b-instruct
- meta/llama-3.3-70b-instruct
- meta/llama-3.1-8b-instruct
- bytedance/seed-oss-36b-instruct
- nvidia/nemotron-3-nano-30b-a3b
- meta/llama-3.2-1b-instruct
- deepseek-ai/deepseek-v3.2
- moonshotai/kimi-k2-thinking
- minimaxai/minimax-m2
- meta/llama-3.2-3b-instruct
- stepfun-ai/step-3.5-flash
- nvidia/llama-3.3-nemotron-super-49b-v1.5
- qwen/qwen3-next-80b-a3b-instruct
- nvidia/llama-3.3-nemotron-super-49b-v1
- openai/gpt-oss-20b
- openai/gpt-oss-120b
- deepseek-ai/deepseek-v3.1-terminus
- qwen/qwen3-next-80b-a3b-thinking


In [ ]:
from langchain.agents import create_agent
from langchain.messages import HumanMessage
from pydantic import BaseModel

class CapitalInfo(BaseModel):
    name: str
    location: str
    vibe: str
    economy: str

model = ChatNVIDIA(model="meta/llama-3.1-70b-instruct") # meta/llama-3.1-70b-instruct, deepseek-ai/deepseek-v3.2
agent = create_agent(
    model=model,
    system_prompt="You are a science fiction writer, create a capital city at the users request.",
    response_format=CapitalInfo
)

question = HumanMessage(content="What is the capital of The Moon?")

response = agent.invoke(
    {"messages": [question]}
)

response["structured_response"]

d:\Softwares\anaconda\envs\twelve\Lib\site-packages\langchain_nvidia_ai_endpoints\chat_models.py:670: UserWarning: Reasoning content was parsed from <think> tags in model output. The tags are currently preserved in the content but will be removed in a future version. The reasoning is available in additional_kwargs['reasoning_content'] and in the reasoning content block in content_blocks. Use reasoning_content instead of parsing tags manually.
  parsed_response = self._custom_postprocess(


Exception: [###] Unknown Error
{'object': 'error', 'message': 'Invalid grammar request with cache hit: key=(\'json\', \'{"type": "array", "minItems": 1, "items": {"type": "object", "anyOf": [{"properties": {"name": {"type": "string", "enum": ["CapitalInfo"]}, "parameters": {"properties": {"name": {"type": "string"}, "location": {"type": "string"}, "vibe": {"type": "string"}, "economy": {"type": "string"}}, "required": ["name", "location", "vibe", "economy"], "type": "object"}}, "required": ["name", "parameters"]}]}}\')', 'type': 'BadRequestError', 'param': None, 'code': 400}

In [19]:
response["structured_response"].name

'Lunar City'

In [26]:
capital_info = response["structured_response"]

capital_name = capital_info.name
capital_location = capital_info.location

print(f"{capital_name} is a city located at {capital_location}")

Lunara Prime is a city located at Northern rim of the Peary Crater, The Moon
